<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


Read data created in the previous chapter.

In [2]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [7]:
df_train.head()
df_valid.head()

,fare_amount,pickuplon,pickuplat,dropofflon,dropofflat,passengers,key
0,6.0,-74.013667,40.713935,-74.007627,40.702992,2,0
1,9.3,-74.007025,40.730305,-73.979111,40.752267,1,1
2,6.9,-73.966400,40.759800,-73.986400,40.762400,1,2
3,36.8,-73.961938,40.773337,-73.865820,40.769607,1,3
4,6.5,-73.989408,40.735895,-73.980600,40.745115,1,4


In [8]:
# TODO: Create an appropriate input_fn to read the training data
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    #ADD CODE HERE
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [9]:
# TODO: Create an appropriate input_fn to read the validation data
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    #ADD CODE HERE
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

Our input function for predictions is the same except we don't provide a label

In [10]:
# TODO: Create an appropriate prediction_input_fn
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    #ADD CODE HERE
     x = df,
     y = None,
     batch_size = 128,
     shuffle = False,
     queue_capacity = 1000
  )

### Create feature columns for estimator

In [11]:
# TODO: Create feature columns
def make_feature_cols():
  return [tf.feature_column.numeric_column(k) for k in FEATURES]

<h3> Linear Regression with tf.Estimator framework </h3>

In [13]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

# TODO: Train a linear regression model
model = tf.estimator.LinearRegressor(feature_columns= make_feature_cols(), model_dir = OUTDIR )

model.train(input_fn = make_train_input_fn(df_train, num_epochs= 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_num_ps_replicas': 0, '_task_id': 0, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd39dac9cc0>, '_save_checkpoints_steps': None, '_is_chief': True, '_model_dir': 'taxi_trained', '_save_summary_steps': 100, '_master': '', '_global_id_in_cluster': 0, '_session_config': None, '_train_distribute': None, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_task_type': 'worker', '_service': None, '_log_step_count_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:step = 1, loss = 194

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [15]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-02-20:46:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-02-20:46:27
INFO:tensorflow:Saving dict for global step 608: average_loss = 108.93364, global_step = 608, loss = 12955.322
RMSE on dataset = 10.437128067016602


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [16]:
# TODO: Predict from the estimator model we trained using test dataset
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([11.292006], dtype=float32)}
{'predictions': array([11.289368], dtype=float32)}
{'predictions': array([11.29026], dtype=float32)}
{'predictions': array([11.287726], dtype=float32)}
{'predictions': array([11.291736], dtype=float32)}
{'predictions': array([11.291509], dtype=float32)}
{'predictions': array([11.290038], dtype=float32)}
{'predictions': array([11.290071], dtype=float32)}
{'predictions': array([11.291895], dtype=float32)}
{'predictions': array([11.289673], dtype=float32)}
{'predictions': array([11.292007], dtype=float32)}
{'predictions': array([11.29218], dtype=float32)}
{'predictions': array([11.285216], dtype=float32)}
{'predictions': array([11.289277], dtype=float32)}
{'predictions': array([11

{'predictions': array([11.288854], dtype=float32)}
{'predictions': array([11.291059], dtype=float32)}
{'predictions': array([11.359754], dtype=float32)}
{'predictions': array([11.291835], dtype=float32)}
{'predictions': array([11.4297695], dtype=float32)}
{'predictions': array([11.292648], dtype=float32)}
{'predictions': array([11.358201], dtype=float32)}
{'predictions': array([11.290084], dtype=float32)}
{'predictions': array([11.291175], dtype=float32)}
{'predictions': array([11.360842], dtype=float32)}
{'predictions': array([11.291347], dtype=float32)}
{'predictions': array([11.288165], dtype=float32)}
{'predictions': array([11.29096], dtype=float32)}
{'predictions': array([11.286573], dtype=float32)}
{'predictions': array([11.362504], dtype=float32)}
{'predictions': array([11.291478], dtype=float32)}
{'predictions': array([11.275563], dtype=float32)}
{'predictions': array([11.288488], dtype=float32)}
{'predictions': array([11.359098], dtype=float32)}
{'predictions': array([11.63862

{'predictions': array([11.360118], dtype=float32)}
{'predictions': array([11.289935], dtype=float32)}
{'predictions': array([11.290523], dtype=float32)}
{'predictions': array([11.289683], dtype=float32)}
{'predictions': array([11.289424], dtype=float32)}
{'predictions': array([11.290976], dtype=float32)}
{'predictions': array([11.361298], dtype=float32)}
{'predictions': array([11.284838], dtype=float32)}
{'predictions': array([11.28915], dtype=float32)}
{'predictions': array([11.288641], dtype=float32)}
{'predictions': array([11.292764], dtype=float32)}
{'predictions': array([11.285684], dtype=float32)}
{'predictions': array([11.429088], dtype=float32)}
{'predictions': array([11.29014], dtype=float32)}
{'predictions': array([11.2909975], dtype=float32)}
{'predictions': array([11.290488], dtype=float32)}
{'predictions': array([11.291303], dtype=float32)}
{'predictions': array([11.360759], dtype=float32)}
{'predictions': array([11.288872], dtype=float32)}
{'predictions': array([11.291673

{'predictions': array([11.290188], dtype=float32)}
{'predictions': array([11.289765], dtype=float32)}
{'predictions': array([11.29057], dtype=float32)}
{'predictions': array([11.2920065], dtype=float32)}
{'predictions': array([11.277329], dtype=float32)}
{'predictions': array([11.27625], dtype=float32)}
{'predictions': array([11.291246], dtype=float32)}
{'predictions': array([11.358466], dtype=float32)}
{'predictions': array([11.290883], dtype=float32)}
{'predictions': array([11.360395], dtype=float32)}
{'predictions': array([11.428977], dtype=float32)}
{'predictions': array([11.36], dtype=float32)}
{'predictions': array([11.2916155], dtype=float32)}
{'predictions': array([11.289508], dtype=float32)}
{'predictions': array([11.360562], dtype=float32)}
{'predictions': array([11.360853], dtype=float32)}
{'predictions': array([11.288688], dtype=float32)}
{'predictions': array([11.293754], dtype=float32)}
{'predictions': array([11.289173], dtype=float32)}
{'predictions': array([11.284977], 

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [17]:
# TODO: Copy your LinearRegressor estimator and replace with DNNRegressor. Remember to add a list of hidden units i.e. [32, 8, 2]
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

# TODO: Train a linear regression model
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2], feature_columns= make_feature_cols(), model_dir = OUTDIR )
model.train(input_fn = make_train_input_fn(df_train, num_epochs= 10))
print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_num_ps_replicas': 0, '_task_id': 0, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd378d08748>, '_save_checkpoints_steps': None, '_is_chief': True, '_model_dir': 'taxi_trained', '_save_summary_steps': 100, '_master': '', '_global_id_in_cluster': 0, '_session_config': None, '_train_distribute': None, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_task_type': 'worker', '_service': None, '_log_step_count_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:step = 1, loss = 340

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [18]:
import google.datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) * 1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) * 1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count * 1.0 AS passengers,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) = {1}".format(base_query, phase)
  else:
    query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), {1}) = {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).execute().result().to_dataframe()

In [19]:
print_rmse(model, df)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-02-20:49:51
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-02-20:49:52
INFO:tensorflow:Saving dict for global step 608: average_loss = 169.34465, global_step = 608, loss = 21516.617
RMSE on dataset = 13.013249397277832


RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License